#### Building a ChatBot with history

In [3]:
import os
from dotenv import load_dotenv
load_dotenv()

groq_api_key=os.getenv("GROQ_API_KEY")

In [5]:
from langchain_groq import ChatGroq
model=ChatGroq(model="Gemma2-9b-It",groq_api_key=groq_api_key)
model

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x000001FBFDC5E6E0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000001FBFDC5D720>, model_name='Gemma2-9b-It', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [6]:
from langchain_core.messages import HumanMessage
model.invoke([HumanMessage(content="Hi, My name is Mudith. I'm a Bioinformatician")])

AIMessage(content="Hi Mudith, \n\nIt's nice to meet you! \n\nWhat kind of bioinformatic work do you do?  I'm always interested in learning more about the fascinating field of bioinformatics. Do you have any particular areas of expertise or projects you're working on that you'd like to talk about? \n\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 73, 'prompt_tokens': 24, 'total_tokens': 97, 'completion_time': 0.132727273, 'prompt_time': 0.000154269, 'queue_time': 0.012982618, 'total_time': 0.132881542}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-5404000c-3ba7-428b-bab2-55ca03802b69-0', usage_metadata={'input_tokens': 24, 'output_tokens': 73, 'total_tokens': 97})

In [7]:
from langchain_core.messages import AIMessage
model.invoke(
    [
        HumanMessage(content="Hi my name is Mudith. I'm a Bioinformatician"),
        AIMessage(content="Hi Mudith, \n\nIt's nice to meet you! \n\nWhat kind of bioinformatic work do you do?  I'm always interested in learning more about the fascinating field of bioinformatics. Do you have any particular areas of expertise or projects you're working on that you'd like to talk about?"),
        HumanMessage(content="Hey What's my name and what do I do?")
    ]
)

AIMessage(content="You said your name is Mudith and that you are a Bioinformatician! \n\nIs there anything else you'd like to tell me about yourself or your work? 😊  \n\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 41, 'prompt_tokens': 113, 'total_tokens': 154, 'completion_time': 0.074545455, 'prompt_time': 0.003598016, 'queue_time': 0.009951993, 'total_time': 0.078143471}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-be5e612b-a5df-47df-a719-4d08dba912d9-0', usage_metadata={'input_tokens': 113, 'output_tokens': 41, 'total_tokens': 154})

##### Message History

In [ ]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store={}

def get_session_history(session_id:str)->BaseChatMessageHistory:
    if session_id not in store:
        store[session_id]=ChatMessageHistory()
    return store[session_id]

with_message_history=RunnableWithMessageHistory(model,get_session_history)

In [8]:
config={"configurable":{"session_id":"chat1"}}